In [1]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
from load_tools import *
from model_tools import *
import pyarrow as pa

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

import warnings
from pandas.errors import PerformanceWarning
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore', category=PerformanceWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

import pandas as pd
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, RFE, mutual_info_regression
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR


In [2]:
SRC_DIR = './07-data-compiled/dataset-ss0110-mnmx-7fg'
FEATURES,LABELS = load_dataset(SRC_DIR)
FEATURES = FEATURES.dropna().iloc[:1000]
LABELS = LABELS.loc[FEATURES.index]['W/L']

FEATURES.shape,LABELS.shape

((1000, 346), (1000,))

In [ ]:
model_stack = ModelStackCV(models=ModelStack.MODELS_CLASSIFICATION, model_params=ModelStack.MODEL_PARAMS_CLASSIFICATION)
predictions = model_stack.fit_predict(FEATURES, LABELS, cv=3)
scores = model_stack.score(predictions, score_names=ModelStack.SCORES_CLASSIFICATION)
print(model_stack.get_params())
scores

In [4]:
from sklearn.feature_selection import f_regression, r_regression


# X, y = make_classification(n_samples=1000, n_features=30)  # Example dataset
# X,y = make_regression(n_samples=1000, n_features=30)  # Example dataset
# X = pd.DataFrame(X)
# y = pd.Series(y)
X,y = FEATURES, LABELS

feature_stack = FeatureStack('classification', mode='percentile', params=[10,20,30])
# feature_stack = FeatureStack([f_regression, mutual_info_regression, r_regression], mode='percentile', params=[10,20,30])

feature_stack.fit(X, y)

scores_all = {}
predictions_all = {}
for transformed_X,meta in feature_stack.transform(X):
    print(meta,transformed_X.shape)  # Prints the shape of each transformed dataset
    # model_stack = ModelStackCV(models=ModelStack.MODELS_CLASSIFICATION, model_params=ModelStack.MODEL_PARAMS_CLASSIFICATION)
    model_stack = ModelStackCV(models='classification', model_params='classification')
    predictions = model_stack.fit_predict(transformed_X, y, cv=3)
    # scores = model_stack.score(predictions, score_names=['accuracy','f1'])
    scores = model_stack.score(predictions, score_names='classification')
    scores_all[meta] = scores
    predictions_all[meta] = predictions

scores_all = pd.concat(scores_all.values(), keys=scores_all.keys(), axis=0)
predictions_all = pd.concat(predictions_all.values(), keys=predictions_all.keys(), axis=1)
scores_all
    

('f_classif', 'percentile', 10) (1000, 35)
('mutual_info_classif', 'percentile', 10) (1000, 35)
('r_regression', 'percentile', 10) (1000, 35)
('f_classif', 'percentile', 20) (1000, 70)
('mutual_info_classif', 'percentile', 20) (1000, 70)
('r_regression', 'percentile', 20) (1000, 70)
('f_classif', 'percentile', 30) (1000, 105)
('mutual_info_classif', 'percentile', 30) (1000, 104)
('r_regression', 'percentile', 30) (1000, 105)


train             
                                                          accuracy        f1   
f_classif           percentile 10 AdaBoostClassifier      0.771503  0.770261  \
                                  KNeighborsClassifier    0.724008  0.723619   
                                  LogisticRegression      0.667501  0.668292   
                                  MLPClassifier           0.846505  0.846699   
                                  RandomForestClassifier  1.000000  1.000000   
                                  RidgeClassifier         0.664004  0.664679   
                                  SVC                     0.742001  0.741681   
mutual_info_classif percentile 10 AdaBoostClassifier      0.774508  0.776075   
                                  KNeighborsClassifier    0.710000  0.712184   
                                  LogisticRegression      0.638004  0.636117   
                                  MLPClassifier           0.953500  0.953717   
                                  RandomForestClassifier  1.000000  1.000000   
                                  RidgeClassifier         0.639505  0.637129   
                                  SVC                     0.828499  0.828674   
r_regression        percentile 10 AdaBoostClassifier      0.771503  0.770261   
...                                                            ...       ...   
f_classif           percentile 30 SVC                     0.800500  0.800512   
mutual_info_classif percentile 30 AdaBoostClassifier      0.823000  0.822449   
                                  KNeighborsClassifier    0.727997  0.728699   
                                  LogisticRegression      0.700001  0.700290   
                                  MLPClassifier           1.000000  1.000000   
                                  RandomForestClassifier  1.000000  1.000000   
                                  RidgeClassifier         0.695001  0.697119   
                                  SVC                     0.816004  0.815270   
r_regression        percentile 30 AdaBoostClassifier      0.816005  0.816429   
                                  KNeighborsClassifier    0.711491  0.709307   
                                  LogisticRegression      0.700491  0.700351   
                                  MLPClassifier           0.998499  0.998492   
                                  RandomForestClassifier  1.000000  1.000000   
                                  RidgeClassifier         0.704986  0.704412   
                                  SVC                     0.800500  0.800512   

                                                                               
                                                         precision    recall   
f_classif           percentile 10 AdaBoostClassifier      0.776234  0.764592  \
                                  KNeighborsClassifier    0.726939  0.720584   
                                  LogisticRegression      0.668036  0.668589   
                                  MLPClassifier           0.847683  0.846277   
                                  RandomForestClassifier  1.000000  1.000000   
                                  RidgeClassifier         0.664863  0.664608   
                                  SVC                     0.744027  0.739434   
mutual_info_classif percentile 10 AdaBoostClassifier      0.772751  0.779476   
                                  KNeighborsClassifier    0.708191  0.716420   
                                  LogisticRegression      0.640861  0.631640   
                                  MLPClassifier           0.951415  0.956056   
                                  RandomForestClassifier  1.000000  1.000000   
                                  RidgeClassifier         0.642875  0.631664   
                                  SVC                     0.829210  0.828248   
r_regression        percentile 10 AdaBoostClassifier      0.776234  0.764592   
...                                                         

In [5]:
{
        str(k):list(v) for k,v in feature_stack.transformed_columns.items()
    }

{"('f_classif', 'percentile', 10)": [('Team_facts_venue_rest_days_tgl_basic',
   'Match Info',
   'H/A'),
  ('Opp_facts_venue_rest_days_tgl_basic', 'Match Info', 'H/A'),
  ('Team_norm_minmax_tgl_basic_stats_cumu_avg', 'Result', 'W/L'),
  ('Team_norm_minmax_tgl_basic_stats_cumu_avg', 'Team', 'FG%'),
  ('Team_norm_minmax_tgl_basic_stats_cumu_avg', 'Team', 'ORB'),
  ('Team_norm_minmax_tgl_basic_stats_cumu_avg', 'Opponent', 'BLK'),
  ('Opp_norm_minmax_tgl_basic_stats_cumu_avg', 'Result', 'W/L'),
  ('Opp_norm_minmax_tgl_basic_stats_cumu_avg', 'Team', 'FG%'),
  ('Opp_norm_minmax_tgl_basic_stats_cumu_avg', 'Team', 'ORB'),
  ('Opp_norm_minmax_tgl_basic_stats_cumu_avg', 'Opponent', 'BLK'),
  ('Team_norm_minmax_tgl_basic_stats_ven_cumu_avg', 'Result', 'W/L'),
  ('Team_norm_minmax_tgl_basic_stats_ven_cumu_avg', 'Team', 'FGA'),
  ('Team_norm_minmax_tgl_basic_stats_ven_cumu_avg', 'Team', 'ORB'),
  ('Team_norm_minmax_tgl_basic_stats_ven_cumu_avg', 'Team', 'TRB'),
  ('Opp_norm_minmax_tgl_basic_stats_

In [26]:
model_stack.get_params()

{'RandomForestRegressor': ('RandomForestRegressor', {'n_estimators': 100}),
 'Ridge': ('Ridge', {}),
 'SVR': ('SVR', {}),
 'LinearRegression': ('LinearRegression', {}),
 'MLPRegressor': ('MLPRegressor', {})}

In [6]:
scores_all.groupby(level=[0,1,2]).mean()

train                     val   
                                               mse        r2           mse   
f_regression           percentile 10  13757.914532  0.591363  16690.838130  \
                                  20   7749.253729  0.769744   9081.532400   
                                  30   6724.412688  0.800355   8006.429677   
mutual_info_regression percentile 10  13753.757030  0.591463  16752.185937   
                                  20   7757.774667  0.769500   9137.455051   
                                  30   8226.828267  0.755549   9710.109256   
r_regression           percentile 10  13721.083863  0.592444  16693.299707   
                                  20   7738.068394  0.770074   9091.644040   
                                  30   6691.233862  0.801304   7967.580869   

                                                
                                            r2  
f_regression           percentile 10  0.501836  
                                  20  0.728817  
                                  30  0.761418  
mutual_info_regression percentile 10  0.500017  
                                  20  0.727097  
                                  30  0.710106  
r_regression           percentile 10  0.501794  
                                  20  0.728579  
                                  30  0.762624

In [9]:
feature_stack.scores

,f_regression,mutual_info_regression,r_regression
0,237.366399,0.084879,0.438341
1,37.368108,0.073717,0.189978
2,306.093920,0.117376,0.484477
3,0.589739,0.011579,0.024302
4,1.738188,0.000000,-0.041697
5,4.370236,0.009771,0.066030
6,20.530699,0.011335,0.141976
7,0.024416,0.008989,0.004946
8,0.083336,0.000000,-0.009138
9,16.337787,0.009992,0.126913


In [14]:
# Get the hex value for current timestamp
hex(int(datetime.now().timestamp()))[2:]

'6570e331'

In [24]:
datetime.now().strftime('%Y%m%d.%H%M%S')

'20231206.151212'

In [6]:
data = load_json('./08-experiments/dataset-ss0110-mnmx-7fg/20231206-174521/specs.json')
data

{'models': {'RandomForestClassifier': ['RandomForestClassifier',
   {'n_estimators': 100}],
  'LogisticRegression': ['LogisticRegression', {'max_iter': 1000}],
  'RidgeClassifier': ['RidgeClassifier', {}],
  'SVC': ['SVC', {}],
  'MLPClassifier': ['MLPClassifier', {}],
  'KNeighborsClassifier': ['KNeighborsClassifier', {}],
  'AdaBoostClassifier': ['AdaBoostClassifier', {}]},
 'features': {"('f_classif', 'percentile', 5)": [['Team_norm_minmax_tgl_basic_stats_cumu_avg',
    'Result',
    'W/L'],
   ['Opp_norm_minmax_tgl_basic_stats_cumu_avg', 'Result', 'W/L'],
   ['Team_norm_minmax_tgl_basic_stats_ven_cumu_avg', 'Result', 'W/L'],
   ['Opp_norm_minmax_tgl_basic_stats_ven_cumu_avg', 'Result', 'W/L'],
   ['Team_norm_minmax_tgl_basic_stats_roll_08_avg', 'Result', 'W/L'],
   ['Opp_norm_minmax_tgl_basic_stats_roll_08_avg', 'Result', 'W/L'],
   ['Team_norm_minmax_tgl_advanced_stats_cumu_avg', 'Result', 'W/L'],
   ['Team_norm_minmax_tgl_advanced_stats_cumu_avg', 'Advanced', 'ORtg'],
   ['Team_n